In [ ]:
%pip install transformers

In [1]:
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd
import csv

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import statistics
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/tess/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load dataset

In [ ]:
df = pd.read_csv("/home/tess/experiments/data_transformations/output_df.csv")

In [ ]:
df.dropna(inplace=True)

In [ ]:
nulls = df[df["category"].str.contains("NULL")]

In [ ]:
df = pd.read_csv("/home/tess/experiments/outputs/categories_inception_raw.csv")

In [ ]:
df["annotation"].value_counts()

In [ ]:
nulls.keys()

In [59]:
df.head()

,Unnamed: 0.1,Unnamed: 0,source_file,sentence,begin,end,annotator,_annotation,aspect,sentence_1,category
0,0,0,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,8,18,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",James Cook,Author: James Cook\nTitle: James Cook\nLanguag...,PERSON
1,1,1,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,68,79,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",NEW ZEALAND,Author: James Cook\nTitle: James Cook\nLanguag...,LOCATION
2,2,2,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,83,94,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",NEW HOLLAND,Author: James Cook\nTitle: James Cook\nLanguag...,LOCATION
3,3,3,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_112-166,117,128,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",New Zealand,"From New Zealand to Australia . ] SUNDAY , 1st...",LOCATION
4,4,4,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_112-166,132,141,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",Australia,"From New Zealand to Australia . ] SUNDAY , 1st...",LOCATION


In [ ]:
df.rename(columns = {"text": "aspect", "_sentence_text": "sentence_1", "annotation": "category"}, inplace = True)

In [ ]:
df.drop(['Unnamed: 0', 'source_file', 'sentence', 'begin', 'end', 'annotator',
       '_annotation'], axis=1, inplace=True)

In [ ]:
df.keys()

In [ ]:
df.dropna(inplace=True)

In [ ]:
len(df)

In [ ]:
#nulls.drop(columns=["sentiment"], inplace=True)
nulls.rename(columns= {"sentence": "sentence_1", "sentence_ID": "sentence"}, inplace=True)

In [ ]:
df.head()

In [ ]:
output_df = pd.concat([df, nulls])

In [ ]:
len(output_df)

In [60]:
output_df["category"].value_counts()

NameError: name 'output_df' is not defined

In [9]:
output_df.to_csv("/home/tess/experiments/outputs/gs_all_aspects_full_null.csv")

NameError: name 'output_df' is not defined

In [55]:
tokenizer = BertTokenizer.from_pretrained("emanjavacas/MacBERTh", truncation = True, max_length=512)
model = BertModel.from_pretrained("emanjavacas/MacBERTh")

# Check for GPU availability

In [26]:
torch.cuda.is_available()

True

In [25]:
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Prepare data for BERT embedding extraction
* Extract embeddings from BERT (BERT vs. MacBerth)
* Take context window of 5 surrounding words into account

In [2]:
#for all aspects + all nulls
df = pd.read_csv("/home/tess/experiments/outputs/gs_all_aspects_full_null.csv")

In [8]:
df.dropna(inplace=True)

In [9]:
df.head()

,Unnamed: 0.1,Unnamed: 0,source_file,sentence,begin,end,annotator,_annotation,aspect,sentence_1,category
0,0,0,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,8,18,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",James Cook,Author: James Cook\nTitle: James Cook\nLanguag...,PERSON
1,1,1,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,68,79,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",NEW ZEALAND,Author: James Cook\nTitle: James Cook\nLanguag...,LOCATION
2,2,2,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_0-111,83,94,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",NEW HOLLAND,Author: James Cook\nTitle: James Cook\nLanguag...,LOCATION
3,3,3,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_112-166,117,128,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",New Zealand,"From New Zealand to Australia . ] SUNDAY , 1st...",LOCATION
4,4,4,GB-117_sample_English_18.txt,GB-117_sample_English_18.txt_112-166,132,141,tess,"w.c.Emotions_Entities(ASP_SENT_NULL=ASPECT, be...",Australia,"From New Zealand to Australia . ] SUNDAY , 1st...",LOCATION


In [32]:
df["new_category"].value_counts()

new_category
PERSON            2103
LOCATION          1672
HUM_LANDFORM      1393
NAT_LANDFORM      1077
FAUNA              488
FLORA              390
NULL               330
WEATHER            305
ORGANISATION       219
NAT_PHENOMENON     208
LAND_COVER         200
BIOME               71
MYTH                43
Name: count, dtype: int64

In [14]:
def ontnull(cat):
    return cat.replace("NULL_", "")

In [10]:
def nullify(cat):
    if "NULL" in cat:
        return "NULL"
    else:
        return cat

In [11]:
df["new_category"] = df["category"].apply(nullify)

In [43]:
df["new_category"].value_counts()

new_category
PERSON            2103
LOCATION          1672
HUM_LANDFORM      1393
NAT_LANDFORM      1077
FAUNA              488
FLORA              390
NULL               330
WEATHER            305
ORGANISATION       219
NAT_PHENOMENON     208
LAND_COVER         200
BIOME               71
MYTH                43
Name: count, dtype: int64

In [35]:
#drop row w/single value
df = df[df.category != "NULL_BIOME"]

In [44]:
tags = list(set(df["new_category"]))
tag2idx = {tag:idx for idx, tag in enumerate(tags)}
idx2tag = {idx:tag for idx, tag in enumerate(tags)}

In [45]:
print(idx2tag)

{0: 'BIOME', 1: 'ORGANISATION', 2: 'LOCATION', 3: 'NAT_LANDFORM', 4: 'WEATHER', 5: 'FLORA', 6: 'NULL', 7: 'FAUNA', 8: 'LAND_COVER', 9: 'NAT_PHENOMENON', 10: 'HUM_LANDFORM', 11: 'MYTH', 12: 'PERSON'}


In [46]:
sentence_list = list(df["sentence_1"])
aspect_list = list(df["aspect"])
tag_list = list(df["new_category"])

In [47]:
t = [[sent, asp, tag] for sent, asp, tag in zip(sentence_list, aspect_list, tag_list)]

In [48]:
#get the position of the aspect in the sentence using the tokenized BERT tokens (zie class)

def get_pos(sent_list,aspect_list):
    first_pos = sent_list.index(aspect_list[1]) #first position bc [0] = [CLS] token
    final_pos = []
    for i in range(0,len(aspect_list)-2):
        final_pos.append(first_pos+i)
    return final_pos

In [49]:
class PosDataset(data.Dataset):
    def __init__(self, tagged_sents):
        sents, aspects, tags = [], [], [] # list of lists
        bugged = 0
        for sent in tagged_sents:
            try:
                sent_tokens = tokenizer.encode(sent[0])
                aspect_tokens = tokenizer.encode(sent[1])
                pos_aspects = get_pos(sent_tokens, aspect_tokens)
                tag = sent[2]
                sents.append(sent_tokens)
                aspects.append(pos_aspects)
                tags.append(tag)
            except:
                bugged+=1
        print("Ignoring {} Buggy Annotations".format(bugged))
        self.sents, self.aspects, self.tags = sents, aspects, tags

    def __len__(self):
        return len(self.sents)

    def __getitem__(self, idx):
        words, aspects, tags = self.sents[idx], self.aspects[idx], tag2idx[self.tags[idx]] # words, tags: string list
        return words, aspects, tags

In [56]:
class Net(nn.Module):
    def __init__(self, vocab_size=None):
        super().__init__()
        self.device = device
        self.model = BertModel.from_pretrained("emanjavacas/MacBERTh", output_hidden_states=True)
        self.fc = nn.Linear(768, vocab_size).to(self.device) #initialize linear model


    def forward(self, sent, aspects, y):
        '''
        x: (N, T). int64
        y: (N, T). int64
        '''
        sent = torch.LongTensor(sent).to(self.device)
        aspects = torch.LongTensor(aspects).to(self.device)
        y = torch.LongTensor(y).to(self.device)

        #print(sent)
        #print(aspects)
        #print(y)

        input_ids = sent.unsqueeze(0)  # Batch size 1

        with torch.no_grad():
            outputs = self.model(input_ids)
            last_hidden_states = outputs.last_hidden_state[0]  # Get last hidden state (embeddings)
            start = 0
            end = len(last_hidden_states)-1
            context_window = 5 # define context window we're interested in (5 before and 5 after)

            if aspects[0]-context_window>0:
                start = aspects[0]-context_window
            if aspects[-1]+context_window<len(last_hidden_states)-1:
                end = aspects[-1]+context_window

            all_aspects = []
            for i in range(start,end):
                all_aspects.append(i)

#For each index in all_aspects, it assigns the corresponding BERT embedding from the last_hidden_states tensor to the corresponding row in the bert_embeds tensor.
#Each row of bert_embeds now holds the BERT embedding for the corresponding aspect.

            bert_embeds = torch.zeros(len(all_aspects),768).to(self.device)
            for i, aspect in enumerate(all_aspects):
                bert_embeds[i] = last_hidden_states[aspect]

#  calculates the mean of the embeddings along axis 0 (the rows).
#  This is done to obtain a single aggregated BERT embedding that represents the information from the context window around the aspects.

            bert_embeds = torch.mean(bert_embeds, axis=0).to(self.device)

        logits = self.fc(bert_embeds)

        y_hat = logits.argmax(-1)
        return logits, y, y_hat, bert_embeds

In [57]:
def train(model, iterator, optimizer, criterion):
    model.train()
    running_loss = 0
    for i, batch in enumerate(iterator):
        words, aspects, y = batch
        _y = y # for monitoring
        optimizer.zero_grad()
        logits, y, _ , _= model(words, aspects, y) # logits: (N, T, VOCAB), y: (N, T)

        logits = logits.view(-1, logits.shape[-1]) # (N*T, VOCAB)
        y = y.view(-1)  # (N*T,)

        loss = criterion(logits, y)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i%100==0: # monitoring
            print("step: {}, loss: {}".format(i, running_loss/100))
            running_loss = 0.0

In [58]:
def eval(model, iterator):
    model.eval()

    Words, Aspects, Y, Y_hat = [], [], [], [],
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            words, aspects, y = batch

            _, _, y_hat, _ = model(words, aspects, y)  # y_hat: (N, T) = predicted labels
            print(y_hat)

            Words.extend(words)
            Aspects.extend(aspects)
            Y.extend(y.numpy().tolist())
            Y_hat.extend([y_hat.cpu().numpy().tolist()])

    ## calc metric
    print(classification_report(Y, Y_hat))

In [59]:
train_dataset = PosDataset(t)
train_iter = data.DataLoader(dataset=train_dataset,
                             batch_size=1,
                             shuffle=False,
                             num_workers=0,
                             pin_memory=False)

from transformers import AdamW
optimizer = AdamW(model.parameters(), lr = 1e-4)

criterion = nn.CrossEntropyLoss()

Ignoring 0 Buggy Annotations


In [60]:
model = Net(vocab_size=len(tag2idx))
model.to(device)

Net(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)

In [61]:
eval(model, train_iter)

tensor(11, device='cuda:0')
tensor(7, device='cuda:0')
tensor(7, device='cuda:0')
tensor(3, device='cuda:0')
tensor(11, device='cuda:0')
tensor(3, device='cuda:0')
tensor(11, device='cuda:0')
tensor(11, device='cuda:0')
tensor(11, device='cuda:0')
tensor(11, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(7, device='cuda:0')
tensor(7, device='cuda:0')
tensor(11, device='cuda:0')
tensor(11, device='cuda:0')
tensor(11, device='cuda:0')
tensor(0, device='cuda:0')
tensor(7, device='cuda:0')
tensor(11, device='cuda:0')
tensor(7, device='cuda:0')
tensor(11, device='cuda:0')
tensor(3, device='cuda:0')
tensor(11, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(11, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(11, device='cuda:0')
tensor(11, device='cuda:0')
tensor(7, device='cuda:0')
tensor(7, device='cuda:0')
tensor(11, device='cuda:0')
tensor(11, d

/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
create_data = []
import tqdm
for i, batch in tqdm.tqdm(enumerate(train_iter)):
        words, aspects, y = batch
        _y = y # for monitoring
        optimizer.zero_grad()
        logits, y, _, bert_embeds = model(words, aspects, y) # logits: (N, T, VOCAB), y: (N, T)

        logits = logits.view(-1, logits.shape[-1]) # (N*T, VOCAB)
        y = y.view(-1)  # (N*T,)

        bert_embeds = bert_embeds.cpu().numpy()
        y = int(y.cpu().numpy()[0])
        create_data.append([bert_embeds, y])

9it [00:00, 83.70it/s]

8499it [01:58, 71.77it/s]


In [31]:
create_data[0]

[array([ 7.53647415e-03,  4.18147892e-01,  5.05546518e-02, -6.76082075e-01,
         5.39479315e-01, -5.26105426e-02,  7.43497849e-01,  5.59271336e-01,
        -1.93149537e-01, -3.37438166e-01, -1.03019672e-02, -5.52356958e-01,
        -4.16336134e-02,  1.35958105e-01, -4.24462378e-01,  8.17599654e-01,
         1.07884258e-01,  2.40996685e-02, -7.99566269e-01,  1.26498953e-01,
         4.21856433e-01,  6.60378113e-02, -1.64153308e-01,  4.49988157e-01,
         8.44218135e-01, -3.36956792e-02,  2.44327579e-02,  9.42802951e-02,
         1.38963625e-01,  6.81205317e-02,  7.09570408e-01,  1.28456622e-01,
        -1.66131649e-02, -3.51224691e-02,  6.52290732e-02, -8.44740048e-02,
        -2.80066550e-01, -1.33265788e-02, -3.28589767e-01,  4.80467737e-01,
        -9.14177597e-01, -1.52425766e-01,  6.74465001e-01, -3.49055827e-01,
         5.23826480e-02, -2.58383065e-01, -8.43550637e-02,  2.72666872e-01,
         1.78080663e-01, -2.28559524e-01,  4.55135554e-01, -6.77296743e-02,
        -1.5

In [63]:
#Dump embeddings to a pickle file
pickle.dump(create_data, open('/home/tess/experiments/outputs/embeddings_MacBerth_category_gs_all_aspects_null_one_cat.pkl', 'wb'))

In [ ]:
X = []
Y = []
for dat in create_data:
    X.append(dat[0])
    Y.append(dat[1])

In [ ]:
print(create_data)

# Train classifiers on embeddings
* 6680 training samples

In [64]:
#to train!
pickled_embeds = pd.read_pickle(r'/home/tess/experiments/outputs/embeddings_MacBerth_category_gs_all_aspects_null_one_cat.pkl')

In [34]:
len(pickled_embeds)

8499

In [65]:
#split embeds + y_variable into X and Y
X = []
Y = []
for dat in pickled_embeds:
  if dat[1] not in [20, 19]:
    X.append(dat[0])
    Y.append(dat[1])

In [35]:
#split embeds + y_variable into X and Y
X = []
Y = []
for dat in pickled_embeds:
  X.append(dat[0])
  Y.append(dat[1])

In [ ]:
#target_names = [v for v in tag2idx.keys() if v not in [20, 19]]

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, shuffle=True, random_state=42, stratify=Y)

In [67]:
names = ["Linear SVM", "MLP Classifier", "Random Forest Classifier", "ADABOOST"]

classifiers = [
    SVC(kernel="linear", C=0.025),

    MLPClassifier(alpha=1, max_iter=1000),

    RandomForestClassifier(n_estimators= 100),

    AdaBoostClassifier(n_estimators=50),

    ]

for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)

    target_names = [idx2tag[el] for el in set(y_test) if el in idx2tag] #get target names from y_test

    print("CLASSIFIER: {}".format(name))

    print(classification_report(y_test, predicted, target_names = target_names))

CLASSIFIER: Linear SVM
                precision    recall  f1-score   support

         BIOME       0.67      0.14      0.24        14
  ORGANISATION       0.64      0.20      0.31        44
      LOCATION       0.63      0.71      0.67       334
  NAT_LANDFORM       0.61      0.67      0.64       215
       WEATHER       0.72      0.64      0.68        61
         FLORA       0.77      0.82      0.80        78
          NULL       0.07      0.03      0.04        66
         FAUNA       0.73      0.62      0.67        98
    LAND_COVER       0.58      0.28      0.37        40
NAT_PHENOMENON       0.47      0.48      0.47        42
  HUM_LANDFORM       0.66      0.65      0.66       279
          MYTH       1.00      0.25      0.40         8
        PERSON       0.76      0.87      0.81       421

      accuracy                           0.67      1700
     macro avg       0.64      0.49      0.52      1700
  weighted avg       0.66      0.67      0.65      1700

CLASSIFIER: MLP Classi

/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CLASSIFIER: ADABOOST
                precision    recall  f1-score   support

         BIOME       0.00      0.00      0.00        14
  ORGANISATION       0.00      0.00      0.00        44
      LOCATION       0.35      0.45      0.39       334
  NAT_LANDFORM       0.29      0.46      0.36       215
       WEATHER       0.33      0.10      0.15        61
         FLORA       0.50      0.12      0.19        78
          NULL       0.00      0.00      0.00        66
         FAUNA       0.29      0.02      0.04        98
    LAND_COVER       0.00      0.00      0.00        40
NAT_PHENOMENON       0.25      0.02      0.04        42
  HUM_LANDFORM       0.13      0.03      0.04       279
          MYTH       0.00      0.00      0.00         8
        PERSON       0.38      0.75      0.51       421

      accuracy                           0.35      1700
     macro avg       0.19      0.15      0.13      1700
  weighted avg       0.28      0.35      0.27      1700



/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/tess/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# k-fold cross-validation

* Mean F1_weighted (Bc classes are uneven)

In [ ]:
classifiers = [SVC(kernel="linear", C=0.025), MLPClassifier(alpha=1, max_iter=1000), RandomForestClassifier(n_estimators= 100), AdaBoostClassifier(n_estimators=50)]

for name, clf in zip(names, classifiers):
  scores = cross_validate(clf, X_train, y_train, cv=10, scoring='f1_weighted', return_estimator=True)
  print(name)
  print(scores['test_score'])
  print(f"Mean f1 macro across validations {statistics.mean(scores['test_score'])}")

In [69]:
classifiers = [SVC(kernel="linear", C=0.025), MLPClassifier(alpha=1, max_iter=1000), RandomForestClassifier(n_estimators= 100), AdaBoostClassifier(n_estimators=50)]

for name, clf in zip(names, classifiers):
  scores = cross_validate(clf, X_train, y_train, cv=10, scoring='f1_macro', return_estimator=True)
  print(name)
  print(scores['test_score'])
  print(f"Mean f1 macro across validations {statistics.mean(scores['test_score'])}")

Linear SVM
[0.53737788 0.436859   0.49722954 0.47427967 0.48831132 0.45600558
 0.53905611 0.50500122 0.45844523 0.48382395]
Mean f1 macro across validations 0.4876389510131997
MLP Classifier
[0.51196534 0.46850474 0.49984481 0.45004584 0.50036814 0.49865148
 0.50304144 0.48605739 0.49712316 0.47313339]
Mean f1 macro across validations 0.4888735728352533
Random Forest Classifier
[0.41390035 0.41391408 0.42145635 0.37353138 0.45993758 0.3900953
 0.41721894 0.40773559 0.38046995 0.38470745]
Mean f1 macro across validations 0.40629669649428624
ADABOOST
[0.12080884 0.14662892 0.10985531 0.13143594 0.12972001 0.14600258
 0.13449625 0.14550013 0.13115294 0.14238868]
Mean f1 macro across validations 0.1337989590356588


# Optimize best models


1.   MLP classifier


- Hyperparameter tuning + 5-fold crossval



In [71]:
mlp_gs = MLPClassifier(alpha=1, max_iter=1000)

parameter_space = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)

/home/tess/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/tess/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/tess/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/tess/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/tess/.local/lib/python3.8/site-packages/sklearn/ne

GridSearchCV(cv=5, estimator=MLPClassifier(alpha=1, max_iter=1000), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(10, 30, 10), (20,)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

In [74]:
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive', 'solver': 'sgd'}


In [ ]:
print(clf.best_score_)

In [72]:
grid_predictions = clf.predict(X_test)

# print classification report
target_names = [idx2tag[el] for el in set(y_test) if el in idx2tag] #get target names from y_test
print(classification_report(y_test, grid_predictions, target_names = target_names))

                precision    recall  f1-score   support

         BIOME       0.80      0.29      0.42        14
  ORGANISATION       0.75      0.34      0.47        44
      LOCATION       0.61      0.63      0.62       334
  NAT_LANDFORM       0.57      0.60      0.59       215
       WEATHER       0.62      0.74      0.68        61
         FLORA       0.74      0.81      0.77        78
          NULL       0.05      0.05      0.05        66
         FAUNA       0.79      0.64      0.71        98
    LAND_COVER       0.47      0.38      0.42        40
NAT_PHENOMENON       0.54      0.45      0.49        42
  HUM_LANDFORM       0.68      0.67      0.67       279
          MYTH       0.00      0.00      0.00         8
        PERSON       0.79      0.85      0.82       421

      accuracy                           0.66      1700
     macro avg       0.57      0.50      0.52      1700
  weighted avg       0.65      0.66      0.65      1700



In [70]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, cv = 5) 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits


KeyboardInterrupt: 

# Pickle best model

In [ ]:
# pickle.dump(clf, open('deus_ex_machina/models/SVM_Coarse_Category.pkl', 'wb'))
clf = pickle.load(open('deus_ex_machina/models/SVM'))